In [1]:
from hdx.utilities.easy_logging import setup_logging
from hdx.hdx_configuration import Configuration
from hdx.data.dataset import Dataset
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords
import re
import itertools 
import pickle
from sklearn.neural_network import MLPClassifier
from fastText import load_model
from sklearn.model_selection import train_test_split
from nltk import ngrams
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
df = pd.read_csv("try.csv")

In [3]:
from nltk.corpus import stopwords
nltk.download('stopwords')
stopWords = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ruochen99/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
fasttext_model = 'wiki.en.bin'
fmodel = load_model(fasttext_model)

In [5]:
def lower_cols(lst):
    return [word.lower() for word in lst]


def remove_chars(lst):

    cleaned = [re.sub('\s+', ' ', mystring).strip() for mystring in lst]
    cleaned = [re.sub(r'[[^A-Za-z0-9\s]+]', ' ', mystr) for mystr in cleaned]
    cleaned = [mystr.replace('_', ' ') for mystr in cleaned]
    return cleaned

def clean_cols(data):
    data = lower_cols(data)
    data = remove_chars(data)
    return data

In [110]:
def preprocess(pandas_dataset, df_target):
    if (not pandas_dataset.empty):
    	organization = 'HDX'   #Replace if datasets contains organization
    	headers = list(pandas_dataset.columns.values)
    	headers = clean_cols(headers)
    for i in range(len(headers)):
        try:
            dic = {'Header': headers[i], 
                   'Data': list(pandas_dataset.iloc[1:, i]), 
                   'Relative Column Position': (i+1) / len(pandas_dataset.columns), 
                   'Organization': organization,
                   'Index': i}
            df_target.loc[len(df_target)] = dic
        except:
            raise Exception("Error: arguments not matched")

    df_result = transform_vectorizers(df_target)
    return df_result

def transform_vectorizers(df_target):
    cols = ['Header_embedding', 'Organization_embedded', 'BOW_counts', 'ngrams_counts']
    df = pd.DataFrame(columns = cols)
    long_string = []
    for i in df_target['Data']:
        result_by_tag = word_extract(i)
        holder_list = ''.join(result_by_tag)
        long_string.append(holder_list)
    bag_vectorizer = CountVectorizer()
    corpus = long_string
    X_vecs_bag = bag_vectorizer.fit_transform(corpus)
    df['BOW_counts'] = [item for item in X_vecs_bag.toarray()]
    ngrams = generate_n_grams(df_target['Header'], 3)
    ngrams_vectorizer = CountVectorizer(tokenizer=lambda doc: doc, lowercase=False)
    X_vec_grams = ngrams_vectorizer.fit_transform(ngrams)
    df = df.iloc[:-2,:]
    df['ngrams_counts'] = [item for item in X_vec_grams.toarray()]
    df['Header_embedding'] = df_target['Header'].astype(str).apply(fmodel.get_sentence_vector)
    df['Organization_embedded'] = df_target['Organization'].astype(str).apply(fmodel.get_sentence_vector)
    cols = ['Header_embedding', 'Organization_embedded', 'BOW_counts', 'ngrams_counts']
    df['features_combined'] = df[cols].values.tolist()
    print(df['features_combined'][4])
#     df['features_combined'] = df['features_combined'].apply(lambda x: np.concatenate(x, axis=None))
    df['features_combined'] = df['features_combined'].apply(lambda x: [val for item in x for val in item])
    df.dropna()
    return df

In [83]:
def remove_stop_words(data_lst):
    #remove stopwords from the data including 'the', 'and' etc.
    wordsFiltered = []
    for w in data_lst:
        if w not in stopWords:
            wordsFiltered.append(w)
    return wordsFiltered

def word_extract(row):
    ignore = ['nan']
    no_white = [i.lstrip() for i in row if i not in ignore and not isinstance(i, float)]
    cleaned_text = [w.lower() for w in no_white if w not in ignore]
    return cleaned_text

def allowed_file(filename):
    return '.' in filename and \
           filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS 

from nltk import ngrams
def generate_n_grams(data_lst, n):
    # cleaned = remove_chars(list(data_lst))
    # cleaned = clean_cols(cleaned)
    cleaned = remove_stop_words(data_lst)
    #make sure that n_grams 'refresh' when a new dataset is encountered!!!!   
    return list(ngrams(cleaned, n))

In [111]:
processed_dataset = preprocess(df, 
                pd.DataFrame(columns=['Header','Data','Relative Column Position','Organization','Index']))

[array([  2.47414857e-02,  -5.48960678e-02,   6.00527711e-02,
        -7.03906417e-02,  -4.67367433e-02,  -7.32377395e-02,
         8.01580697e-02,  -1.76433381e-02,  -1.93424094e-02,
        -6.00202754e-02,  -2.23275833e-02,  -9.04171094e-02,
         3.89071852e-02,  -6.47022873e-02,   6.15325430e-03,
        -1.13157243e-01,  -5.76398000e-02,  -1.21198464e-02,
        -2.22601520e-04,   8.93556774e-02,  -1.45886049e-01,
        -6.35377783e-03,  -3.28403227e-02,   6.64590225e-02,
         5.18646315e-02,   3.48556079e-02,   1.13761825e-02,
        -8.12995732e-02,  -5.29367710e-04,   5.38800918e-02,
        -8.24899152e-02,  -2.10784469e-02,  -5.52759394e-02,
         2.61145970e-03,   1.64948087e-02,  -7.81606436e-02,
        -1.30917365e-02,  -4.80476394e-02,   1.35536253e-01,
         1.96997859e-02,  -1.78993661e-02,  -8.05781037e-02,
         6.78841248e-02,   6.06602132e-02,   6.23246245e-02,
         5.10848127e-02,  -4.83746938e-02,  -1.49348965e-02,
        -5.13083972e-02

In [32]:
model = pickle.load(open("model.pkl", "rb"))

/anaconda3/lib/python3.6/site-packages/sklearn/base.py:311: UserWarning: Trying to unpickle estimator LabelBinarizer from version 0.19.0 when using version 0.19.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/base.py:311: UserWarning: Trying to unpickle estimator MLPClassifier from version 0.19.0 when using version 0.19.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [96]:
processed_dataset['features_combined'] = processed_dataset['features_combined'].apply(lambda x: [x])

In [117]:
type(processed_dataset['features_combined'])

pandas.core.series.Series

In [114]:
model.predict(processed_dataset['features_combined'])

ValueError: setting an array element with a sequence.